# Set up environment

In [2]:
# Install Foundry
!pip install foundry-ml

Defaulting to user installation because normal site-packages is not writeable
  Using cached jsonschema-4.3.0-py3-none-any.whl (71 kB)
  Attempting uninstall: jsonschema
    Found existing installation: jsonschema 4.17.3
    Uninstalling jsonschema-4.17.3:
      Successfully uninstalled jsonschema-4.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.19.0 requires jsonschema>=4.17.3, but you have jsonschema 4.3.0 which is incompatible.

[notice] A new release of pip is available: 23.1 -> 23.1.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [7]:
# Import some generally useful modules
from dlhub_sdk.client import DLHubClient
from foundry import Foundry
import pandas as pd

# Data+Model Template: Diffusion data and model

## Get and show diffusion data



* Dataset 19
* Relevant reference: H. J. Lu, N. Zou, R. Jacobs, B. Afflerbach, X. G. Lu, and D. Morgan, Error assessment and optimal cross-validation approaches in machine learning applied to impurity diffusion, Computational Materials Science 169 (July) (2019). https://doi.org/10.1016/j.commatsci.2019.06.010
* Data DOI: 10.18126/uppe-p8p1 


In [4]:
# Load and print out data
f = Foundry(index="mdf")
data = f.load('diffusion_v1.4', globus=False).load_data()
print(data)
X = data['train'][0]  # Features pre-generation
y = data['train'][1]  # Target variable
print(X)
print(y)

{'train': (    Material compositions 1 Material compositions 2
0                        Ag                      Ag
1                        Ag                      Co
2                        Ag                      Cr
3                        Ag                      Cu
4                        Ag                      Fe
..                      ...                     ...
403                      Zr                      Nb
404                      Zr                      Ta
405                      Zr                      Ti
406                      Zr                      Hf
407                      Zr                      Sn

[408 rows x 2 columns],      E_regression_shift
0              0.000000
1             -0.090142
2              0.259139
3             -0.022200
4              0.317672
..                  ...
403           -0.067020
404            0.153850
405            0.248110
406            0.204140
407            0.248040

[408 rows x 1 columns])}
    Material compositions 

## Get and show diffusion model
* Name: Ryan Jacobs
* Email: rjacobs3@wisc.edu
* Relevant reference: H. J. Lu, N. Zou, R. Jacobs, B. Afflerbach, X. G. Lu, and D. Morgan, Error assessment and optimal cross-validation approaches in machine learning applied to impurity diffusion, Computational Materials Science 169 (July) (2019). https://doi.org/10.1016/j.commatsci.2019.06.010
* Date added/last modified in this notebook: 10/18/22
* Foundry hosted data link: https://foundry-ml.org/#/datasets/10.18126%2Fuppe-p8p1
* Foundry hosted data DOI: 10.18126/uppe-p8p1 

In [5]:
# Set servable to work with
# Note that this first line should generate a link and the link "Please Paste your Auth Code Below:" with an box to enter your code from the link.
dlhub_servable = 'rjacobs3_wisc/MASTML_Tutorial_Model_DLHub_v100_2022-10-13'
# Pring out servable info
desc=DLHubClient().describe_servable(dlhub_servable)
#desc["servable"].keys()
print('Servible Name (if availble): ', desc['dlhub']['shorthand_name'])
print('Servible Datacite (if availble): ', desc['datacite']['identifier']['identifier'])
print('Servible DOI (if availble): ', desc['datacite']['identifier']['identifierType'])

Servible Name (if availble):  rjacobs3_wisc/MASTML_Tutorial_Model_2022-10-18_v2
Servible Datacite (if availble):  10.datacite/placeholder
Servible DOI (if availble):  DOI


In [ ]:
# This model takes as input 2 elements, the first is the host and the second is an impurity, and it returns the activation energy for impurity diffusion.
# For example, the input CuAg gives the activation energy of dilute Ag in a Cu host.
X_test = pd.DataFrame({'Material compositions joined': ['CuAl', 'NiAl', 'HeH', 'NiSi', 'PtPd']})
# Info on input format
# DLHubClient().describe_servable(dlhub_servable)
input_dict = {'X_test': X_test, #X_test_Ag_nofeatures
              'X_test_extra': X_test, #X_extra_Ag
             'featurize': True,
             'featurizer': 'ElementalFeatureGenerator',
             'featurize_on': 'Material compositions joined',
              'composition_df': X_test['Material compositions joined'] #X_test_Ag_nofeatures['Material compositions joined']
             }
#dlhub_servable = MY_USERNAME+'/'+model_name
pred_df = DLHubClient().run(name=dlhub_servable, inputs=input_dict, debug=False, validate_input=False)
pred_df